In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam


In [3]:
def csv_to_image(csv_path, image_path, image_size=(512, 512)):
    """
    Convert a CSV file containing polyline data to a PNG image.
    """
    df = pd.read_csv(csv_path, header=None)
    
    # Initialize image
    fig, ax = plt.subplots(figsize=(image_size[0] / 100, image_size[1] / 100), dpi=100)
    ax.set_xlim(0, image_size[0])
    ax.set_ylim(0, image_size[1])
    ax.set_aspect('equal')
    ax.axis('off')

    polylines = {}
    for index, row in df.iterrows():
        polyline_id = int(row[0])
        num_points = int(row[1])
        x_coords = list(map(float, row[2:2+num_points]))
        y_coords = list(map(float, row[2+num_points:2+2*num_points]))

        if len(x_coords) > 1 and len(y_coords) > 1:
            polylines[polyline_id] = (x_coords, y_coords)
    
    for (x_coords, y_coords) in polylines.values():
        if len(x_coords) > 1 and len(y_coords) > 1:
            polygon = Polygon(list(zip(x_coords, y_coords)), closed=False, edgecolor='black')
            ax.add_patch(polygon)

    plt.gca().invert_yaxis()  # Invert y-axis to match image coordinates
    plt.savefig(image_path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)


In [4]:
def load_images_from_folder(folder, size=(512, 512)):
    """
    Load images from a folder and resize them.
    """
    images = []
    for filename in os.listdir(folder):
        if filename.endswith('.png'):
            img_path = os.path.join(folder, filename)
            image = Image.open(img_path).resize(size).convert('RGB')
            images.append(np.array(image))
    return np.array(images)


In [5]:
def create_folders_and_convert_csv(input_csv_folder, output_csv_folder, input_image_folder, output_image_folder):
    """
    Create folders for images and convert CSVs to PNG images.
    """
    os.makedirs(input_image_folder, exist_ok=True)
    os.makedirs(output_image_folder, exist_ok=True)
    
    # Convert input CSVs to input PNG images
    for csv_file in os.listdir(input_csv_folder):
        if csv_file.endswith('.csv'):
            csv_path = os.path.join(input_csv_folder, csv_file)
            image_path = os.path.join(input_image_folder, csv_file.replace('.csv', '.png'))
            csv_to_image(csv_path, image_path)
    
    # Convert output CSVs to output PNG images
    for csv_file in os.listdir(output_csv_folder):
        if csv_file.endswith('.csv'):
            csv_path = os.path.join(output_csv_folder, csv_file)
            image_path = os.path.join(output_image_folder, csv_file.replace('.csv', '.png'))
            csv_to_image(csv_path, image_path)


In [6]:
# Define paths
input_csv_folder = 'input_csv'
output_csv_folder = 'output_csv'
input_image_folder = 'input_images'
output_image_folder = 'output_images'
predictions_folder = 'predictions'

# Create folders and convert CSVs
create_folders_and_convert_csv(input_csv_folder, output_csv_folder, input_image_folder, output_image_folder)


In [7]:
# Load images
x_images = load_images_from_folder(input_image_folder)
y_images = load_images_from_folder(output_image_folder)

# Ensure same number of images and correct shapes
assert x_images.shape[0] == y_images.shape[0], "Number of input and output images must match"
assert x_images.shape[1:] == (512, 512, 3), "Image dimensions must match the specified size"

# Normalize images
x_images = x_images / 255.0
y_images = y_images / 255.0

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_images, y_images, test_size=0.2, random_state=42)


In [8]:
# Model Definition
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(512, 512, 3)),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(3, (3, 3), activation='sigmoid', padding='same')
])

model.compile(optimizer=Adam(), loss='mse')


In [9]:
# Model Training
model.fit(x_train, y_train, epochs=50, batch_size=1, validation_split=0.2)


Epoch 1/50
3/3 [==============================] - 19s 5s/step - loss: 0.1072 - val_loss: 3.6271e-07
Epoch 2/50
3/3 [==============================] - 15s 5s/step - loss: 1.2804e-07 - val_loss: 4.0771e-17
Epoch 3/50
3/3 [==============================] - 14s 5s/step - loss: 1.3590e-17 - val_loss: 0.0000e+00
Epoch 4/50
3/3 [==============================] - 15s 5s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/50
3/3 [==============================] - 14s 5s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/50
3/3 [==============================] - 14s 5s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/50
3/3 [==============================] - 14s 5s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 8/50
3/3 [==============================] - 14s 5s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/50
3/3 [==============================] - 14s 5s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 10/50
3/3 [==============================] - 14s 5s/step - los

In [11]:
# Model Evaluation
loss = model.evaluate(x_test, y_test)
print(f"Test loss: {loss}")

# Model Prediction
predictions = model.predict(x_test)
print(f"Predictions shape: {predictions.shape}")

# Create predictions folder if it does not exist
os.makedirs(predictions_folder, exist_ok=True)

# Save predictions as images
for i, prediction in enumerate(predictions):
    pred_img = (prediction * 255).astype(np.uint8)
    Image.fromarray(pred_img).save(f'{predictions_folder}/prediction_{i}.png')


1/1 [==============================] - 1s 980ms/step - loss: 0.0000e+00
Test loss: 0.0
1/1 [==============================] - 1s 1s/step
Predictions shape: (1, 512, 512, 3)
